# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import azureml
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
ws = Workspace.from_config()

# Choosing a name for experiment
experiment_name = 'AutoML-Experiment'

experiment=Experiment(ws, experiment_name)
#project_folder = './pipeline-project'


# Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

**OVERVIEW :-**

In this project, I used the "Heart Failure Prediction" dataset from Kaggle.The dataset contains 12 clinical features of 299 patients.Here "DEATH EVENT" is the target variable, we need to predict the possible death event occured due to heat failure of the patients.With the help of HyperDrive and AutoML will get to know the death rates of the patients.


**TASK :-** 

The task of this project is to predict the "DEATH_EVENT" of the patients.This problem belongs to classification category.The target variable death event is in the boolean form, with the help of two models will predict the survival rate of the patients due to heart failure.

12 clinical features are :- age, anaemia, creatinine_phosphokinare, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking, time.

Target variable :- death_event


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
found = False
key = "HeartFailuredata"
description_text = "To predict the survival of the patients"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("The dataset is loaded")

if not found:
        # Creating Dataset and register it into Workspace
        example_data = "https://github.com/Sneha22SP/nd00333-capstone/blob/master/starter_file/heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        # Registering Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

The dataset is loaded


In [5]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [6]:
df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [7]:
df[df.isnull().any(axis=1)]

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT


In [8]:
df["DEATH_EVENT"].value_counts()

0    203
1     96
Name: DEATH_EVENT, dtype: int64

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

automl settings: experiment_timeout_minutes - maximum amount of time the experiment can take. I set 20 minutes to save time. 
max_concurrent_iterations - maximum number of parallel runs executed on a Automl Compute cluster.As it should be less than or equal to the number of nodes (5) I set when creating the compute cluster, it is set to 5. The primary metric is Under the Curve Weighted, **AUC_weighted**, to deal with class imbalance as The AUC is an estimate of the probability that a classifier will rank a randomly chosen positive instance higher than a randomly chosen negative instance. For this reason, the AUC is widely thought to be a better measure than a classification error rate based upon a single prior probability or KS statistic threshold.

AutoMLConfig: This is a binary classification task. Heart Failure "dataset" is imported earlier from the registered dataset in Azure Workspace. The target variable which we need to peredict in this experiment is "DEATH_EVENT". To save time and resources, the enable_early_stopping parameter is set to True.


In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [10]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details


TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
# Retrieving and saving the best automl model
automl_best_run, fitted_automl_model = automl_run.get_output() 

run_dict = automl_best_run.get_details()

print("Best Run ID: ", run_dict["runId"])
print("---"*10)
print("AutoML best run AUC_weighted: ", automl_best_run.get_metrics(name="AUC_weighted"))
print("---"*10)
print("Metrics: ", automl_best_run.get_metrics())
print("---"*10)
print("Fitted model and its hyperparameters : ", fitted_automl_model.steps[-1])



Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Best Run ID:  AutoML_40e1c0a8-d132-4964-879c-fc7387625e0c_58
------------------------------
AutoML best run AUC_weighted:  {'AUC_weighted': 0.9202386822898265}
------------------------------
Metrics:  {'AUC_macro': 0.9202386822898264, 'precision_score_micro': 0.849425287356322, 'f1_score_micro': 0.849425287356322, 'average_precision_score_macro': 0.9099599742551107, 'AUC_weighted': 0.9202386822898265, 'average_precision_score_micro': 0.92499397979046, 'average_precision_score_weighted': 0.9332734103249946, 'recall_score_weighted': 0.849425287356322, 'weighted_accuracy': 0.8726849626230496, 'accuracy': 0.849425287356322, 'balanced_accuracy': 0.814181859870361, 'log_loss': 0.424864291256536, 'matthews_correlation': 0.6586729324826095, 'recall_score_macro': 0.814181859870361, 'recall_score_micro': 0.849425287356322, 'precision_score_macro': 0.8475491429496007, 'precision_score_weighted': 0.8633112103039389, 'AUC_micro': 0.919132778438367, 'f1_score_macro': 0.8163841247547395, 'norm_macro_

In [13]:
print("AutoML Run Summary: ", automl_run.summary())

AutoML Run Summary:  [['StackEnsemble', 1, 0.9093750718749003], ['VotingEnsemble', 1, 0.9202386822898265], ['Failed', 3, nan], ['LightGBM', 9, nan], ['RandomForest', 16, 0.9119456066054467], ['XGBoostClassifier', 12, 0.9107445687946261], ['GradientBoosting', 3, 0.909607640670627], ['ExtremeRandomTrees', 11, 0.905236843095882], ['SVM', 1, 0.8679598190964896], ['KNN', 2, 0.7496161006793443], ['SGD', 1, 0.8670809643784473]]


In [14]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)

model = automl_best_run.register_model(model_name = 'automl_best_model', model_path = './outputs/model.pkl')

In [15]:
print(model)#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)

model = automl_best_run.register_model(model_name = 'automl_best_model', model_path = './outputs/model.pkl')

Model(workspace=Workspace.create(name='quick-starts-ws-138239', subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30', resource_group='aml-quickstarts-138239'), name=automl_best_model, id=automl_best_model:1, version=1, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
automl_best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_a126919a8c02e57f7e517a99050e5c42661e3e2ab5db63a0984edbabb70d9f3b_d.txt',
 'azureml-logs/65_job_prep-tvmps_a126919a8c02e57f7e517a99050e5c42661e3e2ab5db63a0984edbabb70d9f3b_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_a126919a8c02e57f7e517a99050e5c42661e3e2ab5db63a0984edbabb70d9f3b_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/105_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [17]:
# Reference: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-existing-model


# Downloading the scoring file 

automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py' , 'score.py')


In [18]:
# Downloading conda environment 
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml' , 'env.yml')

In [19]:
from azureml.core import Model
# Creating the environment
#env=Environment.from_conda_specification(name='env',file_path = "env.yml")
env = Environment.get(workspace=ws, name="AzureML-AutoML")
# Creating an inference config
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

# Deploying the model as a web service to an Azure Container Instance (ACI)
service_name = 'heartfprediction'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

webservice.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [20]:
print(webservice.state)

Healthy


In [21]:
print(webservice.get_logs())

2021-02-09T08:45:46,208804600+00:00 - iot-server/run 
2021-02-09T08:45:46,209816800+00:00 - rsyslog/run 
2021-02-09T08:45:46,227878400+00:00 - gunicorn/run 
2021-02-09T08:45:46,270070100+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
# Reference: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb 
# and endpoint.py file used in Project 2
scoring_uri = "http://2916d686-189a-43c4-9622-777c8f480910.southcentralus.azurecontainer.io/score"
########## 
df = df.drop(columns=["DEATH_EVENT"])

input_data = json.dumps({
    'data': df[0:4].to_dict(orient='records')
})


print(input_data)
    

{"data": [{"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000.0, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time": 4}, {"age": 55.0, "anaemia": 0, "creatinine_phosphokinase": 7861, "diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 263358.03, "serum_creatinine": 1.1, "serum_sodium": 136, "sex": 1, "smoking": 0, "time": 6}, {"age": 65.0, "anaemia": 0, "creatinine_phosphokinase": 146, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 0, "platelets": 162000.0, "serum_creatinine": 1.3, "serum_sodium": 129, "sex": 1, "smoking": 1, "time": 7}, {"age": 50.0, "anaemia": 1, "creatinine_phosphokinase": 111, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 0, "platelets": 210000.0, "serum_creatinine": 1.9, "serum_sodium": 137, "sex": 1, "smoking": 0, "time": 7}]}


In [23]:
with open("data.json", "w") as _f:
    _f.write(input_data)
    
#######################################################    

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
  

In [25]:
print("Deployment state: " + webservice.state)
print("Scoring URI: " + webservice.scoring_uri)
print("Swagger URI: " + webservice.swagger_uri)

Deployment state: Healthy
Scoring URI: http://2916d686-189a-43c4-9622-777c8f480910.southcentralus.azurecontainer.io/score
Swagger URI: http://2916d686-189a-43c4-9622-777c8f480910.southcentralus.azurecontainer.io/swagger.json


In [26]:
print(resp.json()) 

{"result": [1, 1, 1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
# Retrieve logs for this Webservice
webservice.get_logs()


'2021-02-09T08:45:46,208804600+00:00 - iot-server/run \n2021-02-09T08:45:46,209816800+00:00 - rsyslog/run \n2021-02-09T08:45:46,227878400+00:00 - gunicorn/run \n2021-02-09T08:45:46,270070100+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [36]:
# Delete service
webservice.delete()